## Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Проанализируем полученные данные и узнаем влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

## Оглавление
### 1. [Открытие данных ](#1) 
### 2. [Предобработка данных](#2) 
   #### * [Обработка пропущенных значений](#3) 
   #### * [Замена типа данных](#4) 
   #### * [Обработка дубликатов](#5) 
   #### * [Проверка отдельных столбцов](#6)
   #### * [Лемматизация](#7) 
   #### * [Категоризация](#8) 
### 3. [Ответы на вопросы](#9) 
### 4. [Общий вывод](#10) 

### Шаг 1. Откройте файл с данными и изучите общую информацию.<a id="1"></a>




In [1]:
# загурзим наш файл и посмотрим краткую информацию о типах данных, столбцах и их содеражнии:
import pandas as pd
from pymystem3 import Mystem
m = Mystem()

data = pd.read_csv('/datasets/data.csv')
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


In [2]:
# визуально осмотрим  10-х  строк из датафрейма:
data.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


### Вывод

- есть проблемы с кол-вом отработанных дней(отрицательное значение и огромное кол-во дней ~1000лет)
- разный регистр букв в столбце с образованием {Нужно запретить заемщикам самостоятельно вводить данные, лучше пусть выбирают из выпадющего списк}
- есть пропуски данных в столбцах: доход и отработанное время {На мой взгляд, пропуски не случайны, необходимо программно запретить не заполнять все строчки анкеты }

### Шаг 2. Предобработка данных<a id="2"></a>

### Обработка пропусков<a id="3"></a>

In [3]:
# Определим кол-во пропущенных значений:
data.isnull().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

In [4]:
# проверим столбец с образованием:
data['education'].unique()

array(['высшее', 'среднее', 'Среднее', 'СРЕДНЕЕ', 'ВЫСШЕЕ',
       'неоконченное высшее', 'начальное', 'Высшее',
       'НЕОКОНЧЕННОЕ ВЫСШЕЕ', 'Неоконченное высшее', 'НАЧАЛЬНОЕ',
       'Начальное', 'Ученая степень', 'УЧЕНАЯ СТЕПЕНЬ', 'ученая степень'],
      dtype=object)

In [5]:
# приведем все значения к нижнему регистру:
data['education']= data['education'].str.lower()

In [6]:
# узнаем уникальное кол-во ID образования
data['education_id'].value_counts()

1    15233
0     5260
2      744
3      282
4        6
Name: education_id, dtype: int64

In [7]:
# проверим столбец после изменения регистра
data['education'].value_counts()

среднее                15233
высшее                  5260
неоконченное высшее      744
начальное                282
ученая степень             6
Name: education, dtype: int64

In [8]:
# группировка пропущенных значений в столбцах 'total_income' и 'days_employed' по столбцу 'education'
# найдем медианну для столбцов доход и рабочий стаж:
for education in data['education'].unique():
    median_total_income = data.loc[data['education'] == education, 'total_income'].median()
    median_days_employed = data.loc[data['education'] == education, 'days_employed'].median()
    print(education,median_total_income, median_days_employed)
    data.loc[(data['total_income'].isnull()) & (data['education'] == education), 'total_income'] = median_total_income
    data.loc[(data['days_employed'].isnull()) & (data['education'] == education), 'days_employed'] = median_days_employed

высшее 175340.81885544234 -1342.3734317965254
среднее 136478.64324360332 -1184.3271769041914
неоконченное высшее 160115.39864359115 -1046.437583063456
начальное 117137.35282495995 -551.0625610066053
ученая степень 157259.8985551767 -1380.31604147656


In [9]:
# проверим замену пропущенных значений
data.isnull().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

### Вывод

Избавились от пропусков, предложили возможную причину. Требует уточнений у специалистов.

### Замена типа данных<a id="4"></a>

In [10]:
# Заменили отрицательные значения рабочего стажа, применив модуль:
data['days_employed'] = data['days_employed'].abs()

In [11]:
# Поменяли тип данных на целочисленный, потому что стаж указывают в целых числах:
data['days_employed'] = data['days_employed'].astype(int)

In [12]:
# Поменяли тип данных на целочисленный,убрав все значения после запятой:
data['total_income'] = data['total_income'].astype(int)

In [13]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null int64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null int64
purpose             21525 non-null object
dtypes: int64(7), object(5)
memory usage: 2.0+ MB


### Вывод

Применили целочисленый тип данных для столбца с рабочим стажем и избавились от отрицательных значений.

### Обработка дубликатов<a id="5"></a>

In [14]:
# проверим датасет на дубли
data.duplicated().sum()

71

In [15]:
# Удалим дубли и сбросим индексы

data = data.drop_duplicates().reset_index(drop =True)


In [16]:
# Проверим еще раз на наличие дублей
data.duplicated().sum()

0

### Вывод

Проблемы с дублями были не критичны,возможно, они появились при дополнительных заявках на кредит. Необходимо тех.отделу разобраться с этим.

### Проверим отдельные столбцы:<a id="6"></a>
#### Дети

In [17]:
# проверим столбец "дети" на наличие скрытых ошибок:
data['children'].value_counts()

 0     14091
 1      4808
 2      2052
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

In [18]:
# исправим явные ошибки, а именно заменим значение 20 на 2 и -1 на 1:
data.loc[data['children'] == 20, 'children'] = 2
data.loc[data['children'] == -1,'children' ] = 1

In [19]:
#проверим наши исправления и убедимся, что все получилось:
data['children'].value_counts()

0    14091
1     4855
2     2128
3      330
4       41
5        9
Name: children, dtype: int64

#### Возраст заемщика

In [20]:
# проверим столбец с возрастом заемщика
# сразу бросается в глаза три значения, которые явно с ошибкой : 0, 1,2
data['dob_years'].value_counts()

35    616
40    607
41    605
34    601
38    597
42    596
33    581
39    572
31    559
36    554
44    545
29    544
30    537
48    536
37    536
50    513
43    512
32    509
49    508
28    503
45    496
27    493
52    484
56    483
47    477
54    476
46    472
53    459
57    456
58    454
51    446
55    443
59    443
26    408
60    374
25    357
61    354
62    348
63    269
24    264
64    260
23    252
65    193
22    183
66    182
67    167
21    111
0     101
68     99
69     85
70     65
71     56
20     51
72     33
19     14
73      8
74      6
75      1
Name: dob_years, dtype: int64

In [21]:
# вычислим сколько процентов от выборки занимает сумма неправильных значений:
# 1,03% , поэтому можно пренебречь 
print(223/(len(data['dob_years']))*100)

1.0394332059289644


#### Семейный статус

In [22]:
# проверим столбец со статусом семьи:
data['family_status'].value_counts()

женат / замужем          12339
гражданский брак          4151
Не женат / не замужем     2810
в разводе                 1195
вдовец / вдова             959
Name: family_status, dtype: int64

In [23]:
# проверим столбце со семыйным статусом id:
data['family_status_id'].value_counts()

0    12339
1     4151
4     2810
3     1195
2      959
Name: family_status_id, dtype: int64

#### Пол

In [24]:
# проверим на скрытые ошибки столбец пол:
data['gender'].value_counts()

F      14174
M       7279
XNA        1
Name: gender, dtype: int64

In [25]:
# выявился один не понятный артефакт. Проверим всю строчку:
# ничего критичного
display(data[data['gender'] == 'XNA'])

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
10684,0,2358,24,неоконченное высшее,2,гражданский брак,1,XNA,компаньон,0,203905,покупка недвижимости


#### Рабочий статус

In [26]:
# проверим рабочий статус заемщиков:
# все в порядке, артефактов нет
data['income_type'].value_counts()

сотрудник          11084
компаньон           5078
пенсионер           3829
госслужащий         1457
безработный            2
предприниматель        2
в декрете              1
студент                1
Name: income_type, dtype: int64

### Вывод

Проверили все столбцы входящие в датасет:
1) Исправили артефакты в столбцах ДЕТИ, ОБРАЗОВАНИЕ.
2) Выявили незначительные отклонения в столбцах ВОЗРАСТ и ПОЛ.

### Лемматизация<a id="7"></a>

In [27]:
# проведем лемматизацию с помощью библоиотеки  pymystem3 и выделим ключевые цели для кредита:
purpose_unique = data['purpose'].unique()
text = ''.join(purpose_unique)
lemmas = m.lemmatize(text)

from collections  import Counter
Counter(lemmas)

Counter({'покупка': 4,
         ' ': 59,
         'жильяприобретение': 1,
         'автомобилядополнительный': 1,
         'образованиесыграть': 1,
         'свадьбуоперация': 1,
         'с': 5,
         'жильемобразованиен': 1,
         'проведение': 1,
         'свадьбыпокупка': 1,
         'жилье': 2,
         'для': 2,
         'семьипокупка': 1,
         'недвижимостипокупка': 2,
         'коммерческий': 2,
         'жилой': 2,
         'недвижимостистроительство': 1,
         'собственный': 1,
         'недвижимостинедвижимостьстроительство': 1,
         'недвижимостина': 1,
         'подержать': 2,
         'автомобиляна': 1,
         'свой': 3,
         'автомобиляоперация': 1,
         'недвижимостьюстроительство': 1,
         'недвижимостижильеоперация': 1,
         'со': 1,
         'недвижимостьюавтомобилизаняться': 1,
         'образованиемсделка': 1,
         'автомобилемполучение': 1,
         'образованияавтомобильсвадьбаполучение': 1,
         'дополнительный': 1,
   

In [28]:
# создадим новый столбец для каждого заещика с помощью метода apply и наших лемматизированных строк :
data['lem_purpose'] = data['purpose'].apply(m.lemmatize)

In [29]:
#проверим, что действительно все получилось:
data.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,lem_purpose
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,"[покупка, , жилье, \n]"
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,"[приобретение, , автомобиль, \n]"
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,"[покупка, , жилье, \n]"
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,"[дополнительный, , образование, \n]"
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,"[сыграть, , свадьба, \n]"


### Вывод

Ключевые цели для кредита:
1) Недвижимость
2) Покупка авто
3) Образование
4) Свадьба

### Категоризация данных<a id="8"></a>

In [30]:
# Мы уже раннее определили ключевые цели для кредита, осталось лишь каждого заемщика сопоставить со своей целью:
def purpose_type(lem_purpose):
    if 'жилье'  in lem_purpose or 'недвижимость' in lem_purpose :
        return 'Покупка недвижимости'
    if  'образование' in lem_purpose or 'учеба' in lem_purpose:
        return 'Кредит на образование'
    if 'автомобиль'in lem_purpose or 'авто'in lem_purpose:
        return 'Покупка авто'
    if  'свадьба' in lem_purpose or 'сыграть' in lem_purpose:
        return 'Кредит для бракосочетания'
    return 'Цель не определена'


In [31]:
#создадим дополнитеьный столбец ситоговой целью кредита, с помощью метода apply:
data['total_lem_purpose'] = data['lem_purpose'].apply(purpose_type)

In [32]:
#наглядно убедимся, что столбец создался и итоговые цели для каждого заемщика заполнились:
data.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,lem_purpose,total_lem_purpose
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,"[покупка, , жилье, \n]",Покупка недвижимости
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,"[приобретение, , автомобиль, \n]",Покупка авто
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,"[покупка, , жилье, \n]",Покупка недвижимости
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,"[дополнительный, , образование, \n]",Кредит на образование
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,"[сыграть, , свадьба, \n]",Кредит для бракосочетания
5,0,926,27,высшее,0,гражданский брак,1,M,компаньон,0,255763,покупка жилья,"[покупка, , жилье, \n]",Покупка недвижимости
6,0,2879,43,высшее,0,женат / замужем,0,F,компаньон,0,240525,операции с жильем,"[операция, , с, , жилье, \n]",Покупка недвижимости
7,0,152,50,среднее,1,женат / замужем,0,M,сотрудник,0,135823,образование,"[образование, \n]",Кредит на образование
8,2,6929,35,высшее,0,гражданский брак,1,F,сотрудник,0,95856,на проведение свадьбы,"[на, , проведение, , свадьба, \n]",Кредит для бракосочетания
9,0,2188,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425,покупка жилья для семьи,"[покупка, , жилье, , для, , семья, \n]",Покупка недвижимости


In [33]:
# наглядно мы видим, лишь ограниченное кол-во результатов, для того, чтобы иметь 100% уверености, что все итоговые цели заемщиков стоят
# поищем в столбце результаты, которые нас не устраивают:
data[data['total_lem_purpose'] == 'Цель не определена'].count()

children             0
days_employed        0
dob_years            0
education            0
education_id         0
family_status        0
family_status_id     0
gender               0
income_type          0
debt                 0
total_income         0
purpose              0
lem_purpose          0
total_lem_purpose    0
dtype: int64

In [34]:
# выведем сколько же людей имеют общие цели для кредита и какая цель кредита самая популярная:
data['total_lem_purpose'].value_counts()

Покупка недвижимости         10811
Покупка авто                  4306
Кредит на образование         4013
Кредит для бракосочетания     2324
Name: total_lem_purpose, dtype: int64

### Вывод

После всех действий, мы потвердили, что в данной выборке существует четыри основные цели для кредита (недвижимость, образование, авто и свадьба). Больше всего заявок на кредит подали люди, которые стремятся улучшить свое жилищное состояние (10814 человек), на втором месте люди, мечтающие о новом автомобиле (4308 человек), замыкает тройку лидеров цель на образование (4014 человек), а на последнем месте идет бракосочетание (2335 человек).



In [35]:
#Разделим таблицу data на две части:
family_status_log = data[['family_status_id','family_status']]
print(family_status_log.head(10)) 


   family_status_id     family_status
0                 0   женат / замужем
1                 0   женат / замужем
2                 0   женат / замужем
3                 0   женат / замужем
4                 1  гражданский брак
5                 1  гражданский брак
6                 0   женат / замужем
7                 0   женат / замужем
8                 1  гражданский брак
9                 0   женат / замужем


In [36]:
# вторая часть будет  словарем:
family_status_dict = data[['family_status_id', 'total_income', 'total_lem_purpose']]
print(family_status_dict.head(10)) 

   family_status_id  total_income          total_lem_purpose
0                 0        253875       Покупка недвижимости
1                 0        112080               Покупка авто
2                 0        145885       Покупка недвижимости
3                 0        267628      Кредит на образование
4                 1        158616  Кредит для бракосочетания
5                 1        255763       Покупка недвижимости
6                 0        240525       Покупка недвижимости
7                 0        135823      Кредит на образование
8                 1         95856  Кредит для бракосочетания
9                 0        144425       Покупка недвижимости


In [37]:
# уберем дубликаты в словаре:
family_status_dict = family_status_dict.drop_duplicates().reset_index(drop=True)
print(family_status_dict.head()) 

   family_status_id  total_income          total_lem_purpose
0                 0        253875       Покупка недвижимости
1                 0        112080               Покупка авто
2                 0        145885       Покупка недвижимости
3                 0        267628      Кредит на образование
4                 1        158616  Кредит для бракосочетания


### Шаг 3. Ответьте на вопросы<a id="9"></a>

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [38]:
# создадим сводную таблицу, показывающие как наличие детей зависит от наличия задолжности (в процентах):
data_pivot_children = data.pivot_table(index = ['children'], values = 'debt')*100
data_pivot_children.head(10)

,debt
children,
0,7.543822
1,9.165808
2,9.492481
3,8.181818
4,9.756098
5,0.000000


### Вывод

По результам работы можно с увереностью сказать, что при отсутсвии детей возможный не возврат денежных средст составяет 7.5%, в то время когда дети имеются невозвраст составяет около 9%. По статистике получилось, что люди у которых 4 ребенка хуже всех возвращают долги.
Это обусловленно тем, что при наступлении тяжелых времен собственные дети важнее, чем долги перед банком.

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [39]:
# создадим сводную таблицу, показывающее зависимость просрочек по кредиту от семейного статуса (в процентах):
data_pivot_family_status = data.pivot_table(index = ['family_status'], values = 'debt')*100
data_pivot_family_status.sort_values(by = 'debt', ascending =False)

,debt
family_status,
Не женат / не замужем,9.750890
гражданский брак,9.347145
женат / замужем,7.545182
в разводе,7.112971
вдовец / вдова,6.569343


### Вывод

По результам можно сделать несколько выводов:
1) не женатые/не замужнее и люди живущие в гражданском браке согласно статистике хуже вовращают по долгам, возможная причина этого, что люди боятся ответственности или еще слишком молоды.
2) люди, находящиеся в узаконенных отношениях или люди, которые хотя бы раз побывавшие в них отдают по долгам чаще всего лучше, возможно это связано с тем, что люди более ответственно подходят к вопросы взятия кредита и имеют поддержку в финансовом плане со стороны второй половины.

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [40]:
# найдем квантель на 0.25, 0.5, 0.75:
data['total_income'].quantile([0.25,0.5,0.75])

0.25    107623.00
0.50    141147.00
0.75    195813.25
Name: total_income, dtype: float64

In [41]:
# создадим функцию учитывая данные полученные по квантелям 0.25, 0.5, 0.75:
def total_income_type(total_income):
    if total_income <= 107623:
        return 'низкий класс'
    if total_income <= 141147:
        return 'средний класс'
    if total_income < 195813:
        return 'высокий класс'
    return 'сверхвысокий класс' 

In [42]:
# создадим новый столбец, применив метод apply()  и нашу новую функцию:
data['total_income_type'] = data['total_income'].apply(total_income_type)

In [43]:
#проверим результаты:
data.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,lem_purpose,total_lem_purpose,total_income_type
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,"[покупка, , жилье, \n]",Покупка недвижимости,сверхвысокий класс
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,"[приобретение, , автомобиль, \n]",Покупка авто,средний класс
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,"[покупка, , жилье, \n]",Покупка недвижимости,высокий класс
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,"[дополнительный, , образование, \n]",Кредит на образование,сверхвысокий класс
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,"[сыграть, , свадьба, \n]",Кредит для бракосочетания,высокий класс
5,0,926,27,высшее,0,гражданский брак,1,M,компаньон,0,255763,покупка жилья,"[покупка, , жилье, \n]",Покупка недвижимости,сверхвысокий класс
6,0,2879,43,высшее,0,женат / замужем,0,F,компаньон,0,240525,операции с жильем,"[операция, , с, , жилье, \n]",Покупка недвижимости,сверхвысокий класс
7,0,152,50,среднее,1,женат / замужем,0,M,сотрудник,0,135823,образование,"[образование, \n]",Кредит на образование,средний класс
8,2,6929,35,высшее,0,гражданский брак,1,F,сотрудник,0,95856,на проведение свадьбы,"[на, , проведение, , свадьба, \n]",Кредит для бракосочетания,низкий класс
9,0,2188,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425,покупка жилья для семьи,"[покупка, , жилье, , для, , семья, \n]",Покупка недвижимости,высокий класс


In [44]:
# проверим сколько человек попали в тот или иной класс по уровню дохода:
data['total_income_type'].value_counts()

низкий класс          5364
сверхвысокий класс    5364
высокий класс         5363
средний класс         5363
Name: total_income_type, dtype: int64

In [45]:
# создадим сводную таблицу, показывающее зависимость просрочек по кредиту от уровня дохода (в процентах):
data_pivot_total_income = data.pivot_table(index = ['total_income_type'], values = 'debt')*100
data_pivot_total_income.sort_values(by = 'debt', ascending =False)

,debt
total_income_type,
средний класс,8.819690
высокий класс,8.539996
низкий класс,7.960477
сверхвысокий класс,7.140194


### Вывод

Получили следующие результаты:
1. Средний и высокий класс хуже всего отдают по долгам , по причине большого объема информации вокруг на работе и дома,которая позволяет им получать  высокую оплату труда, но отнимает силы и время, поэтому они просто забывают.
2. Сверхвысокий класс лучше всего отдают по долгам, по причине  того, что финансовая сторона вопроса не существенная, добившись таких высот в финансовом плане, они могут расслабиться и не напрягаться обязанностями по дому , а так же есть возможность не работать.


- Как разные цели кредита влияют на его возврат в срок?

In [46]:
# создадим сводную таблицу, показывающее зависимость просрочек по кредиту от цели кредита (в процентах):
data_pivot_total_purpose = data.pivot_table(index = ['total_lem_purpose'], values = 'debt')*100
data_pivot_total_purpose.sort_values(by = 'debt', ascending =False)

,debt
total_lem_purpose,
Покупка авто,9.359034
Кредит на образование,9.220035
Кредит для бракосочетания,8.003442
Покупка недвижимости,7.233373


### Вывод

Покупка автомобиля,согласно, статистике является самой рискованной целью кредита для банка, это можно обяснить тем, что с покупкой автомобиля появлются дополнительные траты , связанные с обслуживанием авто, высокая вероятность кражи авто или попадание в аварию.
Максимально приоритетная цель для банка по выдаче кредита явлется покупка недвижимости, благодаря тому, что эта довольно крупная покупка и люди заранее просчитвают все тонкости и возможности, а так же до конца выплаты кредита документы на квартиру будут находится у банка в залоге, что позволяет достаточно быстро реализовать недвижимость, если клиент перестанет платить.

### Шаг 4. Общий вывод<a id="10"></a>

В  ходе работы мы проанализировал данные представленные кредитным отделом, провели предобработка данных, включая в себя обработку пропущенных значений, замену типа данных, обработку дубликатов, проведение лемматизации и категоризации. Ответили на интересующие вопросы и дали рекомандации для дальнейшей работы и определили портрет благонадежного клиента.

Ключевые рекомандации:

1) Необходимо, устранить возможность заемщику и/или специалистам кредитного отдела заполнять самостоятельно  пункты анкеты. Нужно реализовать возможность выбора из выпадющего списка готовых вариантов. Тем самым, мы устраним  вероятность неправильного заполнения, а именно различный регистр, грамматические/синтаксические ошибки, раличные сокращения, ошибка со знаками и тд.

2) Необходимо, для того, чтобы исключить дубли, привязывать к каждому человеку свой профиль, чтобы когда подается повторно заявка от одного и того же человека система "подтягивала" данные уже из существующей заявки, а  не создавала повторную заявку.

3) Приоритетное направление для развития в банке, является отдел связанные с ипотекой,т.к. по статистике меньше всего случаев невозврата  с данной целью, а так же длительный срок , что позволяет получить повышенную прибыль при оптимальных рисках.

4) Деньги на покупку автомобиля необходимо выдавать, только с максимальной прозрачной кредитной исторей людям, а так же лучше брать ПТС в залог до выплаты кредита, тем самым остается возможность реализовать движимое имущество, если будут просрочки по кредитам, потому что по статистике это самая рискованная для банка цель кредита.

Идеальный клиент будет мужчина или женщина, находящая в офицальных отношениях или хотя бы раз побываший в них, без детей  и цель займа данного клиента будет покупка недвижимости.
